In [ ]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import os

#endpoint to access the gmail message
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def authenticate_gmail():
    creds = None
    
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
   
    if not creds or not creds.valid:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    return creds

creds = authenticate_gmail()
print("Authentication successful")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=522953999698-51fianftr0t84ifv7usggaprumc7n2rs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55094%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=5kZyCb4YdKP4MAWUI6YAbWQwlCD7oU&access_type=offline
Authentication successful


In [ ]:
import requests
import base64
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

 
url = "http://127.0.0.1:1234/v1/chat/completions"
headers = {"Content-Type": "application/json"}
model = "llama-3b"

def summarize_email(content):
    payload = {
        "messages": [
            {
                "role": "user",
                "content": "Summarize the following email content:\n\n" + content
            }
        ],
        "model": model   
    }

    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        summary = response.json()["choices"][0]["message"]["content"]
        return summary
    else:
        print("Error:", response.status_code, response.text)
        return None

 
def fetch_unread_emails():
    creds = Credentials.from_authorized_user_file('token.json', ["https://www.googleapis.com/auth/gmail.readonly"])
    service = build("gmail", "v1", credentials=creds)

 
    results = service.users().messages().list(userId="me", labelIds=["UNREAD"], maxResults=5).execute()
    messages = results.get("messages", [])
    
    if not messages:
        print("No unread messages found.")
        return

    for message in messages:
        msg = service.users().messages().get(userId="me", id=message["id"]).execute()
        for part in msg.get("payload", {}).get("parts", []):
            if part["mimeType"] == "text/plain":
                email_content = base64.urlsafe_b64decode(part["body"]["data"]).decode("utf-8")
                summary = summarize_email(email_content)
                print("Email Summary:\n", summary)

 
fetch_unread_emails()


Email Summary:
 This email appears to be from HP and is promoting their Diwali campaign. Here's a summary of the content:

* HP wishes its customers a happy and prosperous Diwali.
* The company is launching a new story about an elderly man named Sharad who uses technology to connect with his granddaughter and share stories that are valuable to Indian heritage.
* HP encourages recipients to submit their favorite mythological stories or folklore through its website (www.printlearncenter.com/hpdiwali).
* The email promotes various HP products, including laptops, desktops, printers, and monitors, with features such as free and fast delivery, 100% original products, and interest-free EMI options.
* It also mentions HP's commitment to creating technology that makes life better for everyone and provides contact information for customers to reach out with questions or feedback.
* Finally, the email includes links to unsubscribe from future emails and visit HP's website for more information on 